In [1]:
%matplotlib inline
import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
import matplotlib.pyplot as plt
import mpmath

from synthetic.generate import SingleTaskTreeDepsGenerator
from metal.label_model import LabelModel
from metal.label_model.utils import (
    compute_mu,
    compute_covariance,
    compute_inv_covariance,
    print_matrix,
    visualize_matrix
)

from amc_utils import *

/lfs/1/annhe/anaconda3/envs/metal/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Generate synthetic dataset 
np.random.seed(1)
N = 50
M = 4
K = 1
data = SingleTaskTreeDepsGenerator(N, M, k=K, edge_prob=1.0)


(0, 1)   [0.18310474]
(1, 1)   [0.26763419]
(2, 1)   [0.41100465]
(3, 1)   [0.45709073]
((0, 1), 1, 1)   [0.20776335]
((1, 2), 1, 1)   [0.1838389]
((0, 3), 1, 1)   [0.2370439]
Labeler =  0
val:  0
i:  0  y:  1  val:  0
P(L= 0 , Y= 1 ) =  0.38617669571241636
val:  1
i:  0  y:  1  val:  1
P(L= 1 , Y= 1 ) =  0.6138233042875836
Labeler =  1
val:  0
i:  1  y:  1  val:  0
P(L= 0 , Y= 1 ) =  0.37555302648243905
val:  1
i:  1  y:  1  val:  1
P(L= 1 , Y= 1 ) =  0.6244469735175607
Labeler =  2
val:  0
i:  2  y:  1  val:  0
P(L= 0 , Y= 1 ) =  0.37172826124446584
val:  1
i:  2  y:  1  val:  1
P(L= 1 , Y= 1 ) =  0.628271738755534
Labeler =  3
val:  0
i:  3  y:  1  val:  0
P(L= 0 , Y= 1 ) =  0.35418461558314
val:  1
i:  3  y:  1  val:  1
P(L= 1 , Y= 1 ) =  0.6458153844168599
Labelers =  (0, 1)
P(L_ 0 = 0 , L_ 1 = 0  | Y =  1 ) =  0.15663394629197847
P(L_ 0 = 0 , L_ 1 = 1  | Y =  1 ) =  0.22954274942043787
P(L_ 0 = 1 , L_ 1 = 0  | Y =  1 ) =  0.21891908019046055
P(L_ 0 = 1 , L_ 1 = 1  | Y =  1 ) =  0

IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
print(data.p_solo)
print(len(data.p_solo))

### Let's plug this O into our AMC algorithm

In [ ]:
from amc_utils import *

### First iteration of AMC

In [ ]:
# C_synth = O_true - np.outer(mu, mu)
# J_synth = np.linalg.pinv(C_synth)
# J_clean = copy.deepcopy(J_synth)
# # for i in range(M):
# #     for j in range(M):
# #         if abs(J_clean[i,j]) < 0.3:
# #             J_clean[i,j] = 0.0
# # print(J_clean)

# sg_no_diag = np.copy(J_synth)
# # for i in range(M):
# #     sg_no_diag[i*2:i*2+2,i*2:i*2+2] = np.zeros([2,2])
# visualize_matrix(np.abs(sg_no_diag))

# print(iterative_deps_mask)
# max_val, max_ind, J_clean = find_largest(O_true,mu,M,iterative_deps_mask,0.1)
# print("Index of largest value: ", max_ind)
# print("Largest value: ", max_val)

In [ ]:
def amc(O, O_inv,iterative_deps_mask,thresh=0.1, nonzeros=3):
    dim = np.shape(O)[0]
#    iterative_deps_mask = samplegrid(dim,dim,nonzeros)
#     try:
#         C_synth = O - np.outer(mu_true, mu_true)
#         J_synth = np.linalg.pinv(C_synth)
#     except:
#         print("Failed to invert J before loop")
#         return np.zeros(np.shape(mu_true)), iterative_deps_mask
    num_iters = 0
    while(True):
        num_iters = num_iters + 1
        starttime = time.time()
        mu = solveMatrixCompletionWithMu(O_inv,O,iterative_deps_mask)
        max_val, max_ind, J_clean = find_largest(O,mu,dim,iterative_deps_mask,thresh)
        #if max_val < 1e-6: return J_distances, mu_distances, max_vals, mu, num_iters
        if max_val < 1e-6: 
            return mu, iterative_deps_mask

        iterative_deps_mask.append(max_ind)
    #return J_distances, mu_distances, max_vals, mu, num_iters
    return mu, iterative_deps_mask

In [ ]:
O = data.O_true
mu_true = data.mu_true
O_inv = np.linalg.pinv(O)
print("edges: ", data.E)

iterative_deps_mask = [(i,i) for i in range(M)]

In [ ]:
C = O - np.outer(mu_true, mu_true)
J = np.linalg.pinv(C)
print(J)